# data generator

In [7]:
# coding: utf-8
import argparse
from genericpath import exists
import time
import math
import os, sys
import itertools

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

sys.path.append('pytorch')
sys.path.append('pytorch/utils')
from data_utils import *
from mem_transformer import *
from utils.exp_utils import *
from utils.data_parallel import *


In [8]:

# if args.d_embed < 0:
#     args.d_embed = args.d_model

# assert args.ext_len >= 0, 'extended context length must be non-negative'
# assert args.batch_size % args.batch_chunk == 0

# args.work_dir = '{}-{}'.format(args.work_dir, args.dataset)
# args.work_dir = os.path.join(args.work_dir, time.strftime('%Y%m%d-%H%M%S'))
# logging = create_exp_dir(args.work_dir,
#     scripts_to_save=['train.py', 'mem_transformer.py'], debug=args.debug)

# # Set the random seed manually for reproducibility.
# np.random.seed(args.seed)
# torch.manual_seed(args.seed)
# if torch.cuda.is_available():
#     if not args.cuda:
#         print('WARNING: You have a CUDA device, so you should probably run with --cuda')
#     else:
#         torch.cuda.manual_seed_all(args.seed)

# # Validate `--fp16` option
# if args.fp16:
#     if not args.cuda:
#         print('WARNING: --fp16 requires --cuda, ignoring --fp16 option')
#         args.fp16 = False
#     else:
#         try:
#             from apex.fp16_utils import FP16_Optimizer
#         except:
#             print('WARNING: apex not installed, ignoring --fp16 option')
#             args.fp16 = False

# device = torch.device('cuda' if args.cuda else 'cpu')

###############################################################################
# Load data
###############################################################################


In [9]:

class dummy:
    def __init__(self) -> None:
        pass

args = dummy()
args.data = 'data/'
args.dataset = 'enwik8'
args.batch_size = 128
args.tgt_len = 51
device = 'cpu'
args.ext_len = 5


corpus = get_lm_corpus(args.data, args.dataset)

ntokens = len(corpus.vocab)
args.n_token = ntokens

eval_batch_size = 10
# tr_iter = corpus.get_iterator('train', args.batch_size, args.tgt_len,
#     device=device, ext_len=args.ext_len)

Producing dataset enwik8...
building vocab with min_freq=0, max_size=None
final vocab size 1356566 from 1356566 unique tokens


In [10]:
args.dataset, corpus.dataset

('enwik8', 'enwik8')

In [17]:
# def get_iterator(self, split, *args, **kwargs):
self = corpus
split = 'train'

args = {}
kwargs = {'bsz': 128, 'bptt':1}
if split == 'train':
    if self.dataset in ['ptb', 'wt2', 'wt103', 'enwik8', 'text8']:
        data_iter = LMOrderedIterator(self.train, *args, **kwargs)
    elif self.dataset == 'lm1b':
        kwargs['shuffle'] = True
        data_iter = LMMultiFileIterator(self.train, self.vocab, *args, **kwargs)
elif split in ['valid', 'test']:
    data = self.valid if split == 'valid' else self.test
    if self.dataset in ['ptb', 'wt2', 'wt103', 'enwik8', 'text8', 'enwiki8']:
        data_iter = LMOrderedIterator(data, *args, **kwargs)
    elif self.dataset == 'lm1b':
        data_iter = LMShuffledIterator(data, *args, **kwargs)

In [23]:
i, (s, t, l) = list(enumerate(data_iter))[0]

In [25]:
s.shape, t.shape

(torch.Size([1, 128]), torch.Size([1, 128]))

In [1]:
import numpy as np
import time
import torch
import torch
import sys
sys.path.append('pytorch')
sys.path.append('pytorch/utils')

from pytorch.mem_transformer import *
from pytorch import data_utils

from experiment_utils.run_experiment import *
from experiment_utils.generate_data import *

## Variables

In [2]:
from sklearn.model_selection import ParameterGrid

TAG = '10tkn_len24_ext'

TASK_NAME = 'copy'
TRAIN_SIZE = 1000
VAL_SIZE = 200
TEST_SIZE = 100
NUM_INITS = 1


NUM_BATCHES = int(4e5)
BATCH_SIZE = 128
GENERATE_EVERY  = 10000
NUM_TOKENS = 10 + 2
ENC_SEQ_LEN = 24
DEC_SEQ_LEN = 48

INPUT_LEN = 24

#### Generate data

In [3]:
# # !mkdir data24
# np.random.seed(42)

# generator = copy_generator(batch_size=BATCH_SIZE, enc_seq_len=ENC_SEQ_LEN, dec_seq_len=DEC_SEQ_LEN, num_tokens=NUM_TOKENS)
# generate_data(generator, path=f'data{INPUT_LEN}', task_name=TASK_NAME, train_size=TRAIN_SIZE, test_size=TEST_SIZE, val_size=VAL_SIZE, batch_size=BATCH_SIZE)  

In [4]:
class data_loader:
    def __init__(self, task_name, path='data', batch_size=32, none_mask=True):
        self.X, self.y = np.load(f'{path}/{task_name}_X.npy'), np.load(f'{path}/{task_name}_y.npy')
        self.data_size = self.X.shape[0]
        self.data_ptr = 0

        if none_mask:
            self.src_mask, self.tgt_mask = None, None
        else:
            self.src_masks, self.tgt_mask = np.load(f'{path}/{task_name}_mask.npy'), None

        self.batch_size = batch_size
        self.none_mask = none_mask

    def __next__(self):
        X = self.X[self.data_ptr: self.data_ptr+self.batch_size]
        y = self.y[self.data_ptr: self.data_ptr+self.batch_size]
        
        if not self.none_mask:
            sm = self.src_masks[self.data_ptr: self.data_ptr+self.batch_size]
            sm = torch.tensor(sm).cuda()
        else:
            sm = None
            
        self.data_ptr = (self.data_ptr + self.batch_size) % self.data_size

        return torch.tensor(X),\
                torch.tensor(y),\
                sm, self.tgt_mask

### Run

In [5]:
model_parameters = ParameterGrid({
                'n_layer': [2],
                'n_head': [4],
                'd_head': [128],
                'num_mem_tokens': [9, 0], 
                'mem_len': [0]})

param = list(model_parameters)[0]

fixed_parameters = {'n_token': NUM_TOKENS,
                    'd_model': param['d_head'],# + param['num_mem_tokens']-1,
                    'd_inner': param['d_head'],
                    'dropout': 0,
                    'dropatt': 0,
                    'tie_weight': True,
                    'div_val': 1, # ???????
                    'tie_projs': [False],
                    'tgt_len': DEC_SEQ_LEN,
                    'ext_len': 0, 
                    'cutoffs': [],
                    'attn_type': 0,}

model = MemTransformerLM(**param, **fixed_parameters)#.cuda()

gen_train = data_loader(path=f'data{INPUT_LEN}', task_name=f'{TASK_NAME}_train', batch_size=21)
src, tgt, _, _ = next(gen_train)
# src, tgt = src.cuda(), tgt.cuda()
src, tgt = src.cpu().T, tgt.cpu().T

mems = tuple()
# model(src, tgt.contiguous(), *mems)


In [6]:
model.mem_tokens

In [19]:
out, mems = model._forward(src[:, :1], mems=None)

r_head_k tensor([[[-0.1610,  0.2663,  0.7317,  ..., -0.2776,  0.3494,  0.2219],
         [-0.4773,  0.0291, -0.0052,  ...,  0.1036, -0.2353, -0.8195],
         [-0.5486,  0.5329,  0.8749,  ..., -0.0964, -0.0407, -0.5861],
         [-0.2102, -0.8608,  0.5378,  ...,  0.3381,  0.6476,  0.5246]],

        [[-0.1123,  0.2265,  0.7049,  ..., -0.2572,  0.4042,  0.1958],
         [-0.5291, -0.0010, -0.1122,  ...,  0.0622, -0.1191, -0.7520],
         [-0.4054,  0.6335,  0.8337,  ..., -0.1049,  0.0407, -0.6469],
         [-0.0840, -0.8060,  0.4163,  ...,  0.3796,  0.5944,  0.4686]],

        [[-0.1388,  0.2334,  0.6685,  ..., -0.1457,  0.5046,  0.1005],
         [-0.5478,  0.0256, -0.2161,  ...,  0.1366,  0.0666, -0.6371],
         [-0.3217,  0.6339,  0.8744,  ..., -0.2130,  0.0851, -0.6712],
         [-0.0076, -0.6470,  0.3289,  ...,  0.5306,  0.5669,  0.2955]]],
       grad_fn=<SliceBackward>)
r_head_k tensor([[[-0.1321, -0.1172,  0.4521,  ..., -0.1367, -0.1690,  0.0713],
         [-0.5776,  0

/home/ayd98/Desktop/MIPT/TXL/pytorch/mem_transformer.py:279: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1104.)
  attn_score = attn_score.float().masked_fill(


In [21]:
out.shape

torch.Size([33, 1, 128])

In [32]:
out[0][0], out[1][0], out[0][0] - out[1][0] 

(tensor([-1.5600,  1.8477,  0.3658,  0.9825, -0.0475,  1.3295, -0.1483, -1.7067,
          0.6093, -1.0218, -0.1678, -1.3956,  1.8529,  0.7607,  0.5487, -0.6880,
          1.1657, -1.3241, -0.9002, -0.1482, -1.6440,  0.9515,  0.2808,  0.8881,
          0.3642, -1.5069, -0.1935,  0.9324, -0.9401,  0.1613,  1.4500, -1.0660,
         -0.4886,  0.6992,  0.9827,  1.3505,  0.0044,  0.8011,  0.8062, -0.0596,
         -0.8604,  0.5002,  0.0251, -1.0938,  2.0272, -1.1475, -0.7047, -0.2692,
          0.0681, -0.5827,  0.3875,  0.2979,  0.9092,  1.3300, -1.9834,  1.0851,
         -1.7017, -0.6386,  0.3707,  0.4793, -1.6834,  0.1440, -1.2206,  0.6837,
          0.6079, -1.7838, -0.2298, -1.5133, -3.4377, -0.6613,  0.1642, -0.2854,
         -1.0116, -0.2610, -1.0397, -0.0208,  0.5359,  0.5490,  0.0329,  2.2298,
         -0.3427,  0.2675,  0.5686,  1.7059, -0.0446, -0.6895, -0.4770, -0.4737,
         -1.1829, -0.1803, -0.3275, -1.0151,  0.6898, -1.0112,  0.7736, -0.0343,
          0.5162, -0.5593,  

In [25]:
out[0][0] - out[1][0]

tensor([ 2.3842e-07,  0.0000e+00,  5.9605e-08,  0.0000e+00, -4.2468e-07,
         1.1921e-07, -4.4703e-08,  1.1921e-07,  0.0000e+00,  1.1921e-07,
        -1.9372e-07,  1.1921e-07,  1.1921e-07,  5.9605e-08,  1.7881e-07,
         0.0000e+00,  1.1921e-07,  1.1921e-07, -1.1921e-07,  1.4901e-08,
         1.1921e-07,  0.0000e+00,  2.9802e-08, -1.7881e-07,  2.9802e-08,
         1.1921e-07,  2.2352e-07, -1.1921e-07, -5.9605e-08,  1.3411e-07,
        -1.1921e-07,  2.3842e-07, -1.1921e-07,  5.9605e-08, -5.9605e-08,
         1.1921e-07,  9.5926e-08,  0.0000e+00,  1.7881e-07, -1.6019e-07,
        -5.9605e-08,  5.9605e-08,  2.7940e-08, -2.3842e-07,  0.0000e+00,
         0.0000e+00,  3.5763e-07,  8.9407e-08,  2.2352e-08, -1.1921e-07,
        -2.9802e-08, -2.9802e-08,  5.9605e-08,  1.1921e-07,  0.0000e+00,
         2.3842e-07,  1.1921e-07, -5.9605e-08,  5.9605e-08,  1.7881e-07,
         1.1921e-07,  1.0431e-07,  0.0000e+00,  1.7881e-07, -1.1921e-07,
        -2.3842e-07,  2.3842e-07, -1.1921e-07,  2.3

In [8]:
mem_tokens = None
dec_inp = src[:, :2]

self = model
mems = None
mem_tokens = None
model.init_mem_tokens()
# 


In [9]:

word_emb = self.word_emb(dec_inp)

mlen = mems[0].size(0) if mems is not None else 0

# Concat with mem_tokens
if mem_tokens is not None:
    word_emb = torch.cat((mem_tokens.detach(), word_emb), dim=0)
elif self.num_mem_tokens not in (0, None):
    mem_tokens = self.mem_tokens.reshape(self.num_mem_tokens, 1, -1).repeat(1, dec_inp.shape[1], 1)
    word_emb = torch.cat((mem_tokens.detach(), word_emb), dim=0)

# qlen, bsz = dec_inp.size()
qlen = word_emb.shape[0]
klen = mlen + qlen
if self.same_length:
    all_ones = word_emb.new_ones(qlen, klen)
    mask_len = klen - self.mem_len
    if mask_len > 0:
        mask_shift_len = qlen - mask_len
    else:
        mask_shift_len = qlen
    dec_attn_mask = (torch.triu(all_ones, 1+mlen)
            + torch.tril(all_ones, -mask_shift_len)).byte()[:, :, None] # -1
else:
    dec_attn_mask = torch.triu(
        word_emb.new_ones(qlen, klen), diagonal=1+mlen).byte()[:,:,None]

hids = []
if self.attn_type == 0: # default
    pos_seq = torch.arange(klen-1, -1, -1.0, device=word_emb.device, 
                            dtype=word_emb.dtype)
    if self.clamp_len > 0:
        pos_seq.clamp_(max=self.clamp_len)
    pos_emb = self.pos_emb(pos_seq)

    core_out = self.drop(word_emb)
    pos_emb = self.drop(pos_emb)

    hids.append(core_out)
    # for i, layer in enumerate(self.layers):
    #     mems_i = None if mems is None else mems[i]
    #     core_out = layer(core_out, pos_emb, self.r_w_bias,
    #             self.r_r_bias, dec_attn_mask=dec_attn_mask, mems=mems_i)
    #     hids.append(core_out)

In [10]:
start = core_out
start.shape

torch.Size([33, 2, 128])

In [11]:
w, r, r_w_bias, r_r_bias, attn_mask, mems = start, pos_emb, self.r_w_bias, self.r_r_bias,  None, None

In [13]:
res = model.layers[0].dec_attn(start, pos_emb, self.r_w_bias, self.r_r_bias,  None, None)

r_head_k tensor([[[-0.1610,  0.2663,  0.7317,  ..., -0.2776,  0.3494,  0.2219],
         [-0.4773,  0.0291, -0.0052,  ...,  0.1036, -0.2353, -0.8195],
         [-0.5486,  0.5329,  0.8749,  ..., -0.0964, -0.0407, -0.5861],
         [-0.2102, -0.8608,  0.5378,  ...,  0.3381,  0.6476,  0.5246]],

        [[-0.1123,  0.2265,  0.7049,  ..., -0.2572,  0.4042,  0.1958],
         [-0.5291, -0.0010, -0.1122,  ...,  0.0622, -0.1191, -0.7520],
         [-0.4054,  0.6335,  0.8337,  ..., -0.1049,  0.0407, -0.6469],
         [-0.0840, -0.8060,  0.4163,  ...,  0.3796,  0.5944,  0.4686]],

        [[-0.1388,  0.2334,  0.6685,  ..., -0.1457,  0.5046,  0.1005],
         [-0.5478,  0.0256, -0.2161,  ...,  0.1366,  0.0666, -0.6371],
         [-0.3217,  0.6339,  0.8744,  ..., -0.2130,  0.0851, -0.6712],
         [-0.0076, -0.6470,  0.3289,  ...,  0.5306,  0.5669,  0.2955]]],
       grad_fn=<SliceBackward>)


In [14]:
res

tensor([[[-0.4400, -0.2063, -0.1463,  ...,  0.3321, -1.0940, -0.4041],
         [-0.8133,  0.4882, -0.3307,  ...,  0.4120, -1.5649,  0.0682]],

        [[-0.4213, -0.2204, -0.1544,  ...,  0.3347, -1.0822, -0.4061],
         [-0.8187,  0.4928, -0.3231,  ...,  0.4113, -1.5746,  0.0631]],

        [[-0.3957, -0.2444, -0.1613,  ...,  0.3342, -1.0687, -0.4108],
         [-0.8310,  0.4951, -0.3280,  ...,  0.3992, -1.5953,  0.0467]],

        ...,

        [[-0.1819, -0.6903,  0.4016,  ...,  0.3736,  0.2657, -1.9263],
         [ 0.6139, -0.6959, -1.4468,  ...,  0.5378, -0.5685, -1.7276]],

        [[-1.0107, -0.7791, -2.1517,  ...,  1.3996,  0.1962,  0.2709],
         [-1.3758,  0.0292, -1.4295,  ..., -0.0346, -0.0155, -0.0834]],

        [[-1.6117, -1.1003, -0.2831,  ..., -0.9778, -1.4025,  0.3320],
         [-0.2470, -0.1687,  0.0569,  ...,  0.2756,  0.2057, -0.2320]]],
       grad_fn=<NativeLayerNormBackward>)

In [ ]:

r_net = nn.Linear(self.d_model, self.n_head * self.d_head, bias=False)
qkv_net = nn.Linear(d_model, 3 * n_head * d_head, bias=False)

In [34]:
qlen, rlen, bsz = w.size(0), r.size(0), w.size(1)

if mems is not None:
    cat = torch.cat([mems, w], 0)
    if self.pre_lnorm:
        w_heads = self.qkv_net(self.layer_norm(cat))
    else:
        w_heads = self.qkv_net(cat)
    r_head_k = self.r_net(r)
    # print(r_head_k.shape, w.shape, cat.shape, r_head_k)

    w_head_q, w_head_k, w_head_v = torch.chunk(w_heads, 3, dim=-1)
    w_head_q = w_head_q[-qlen:]
else:
    if self.pre_lnorm:
        w_heads = self.qkv_net(self.layer_norm(w))
    else:
        w_heads = self.qkv_net(w)
    r_head_k = self.r_net(r)
    # print(r_head_k.shape, w.shape, r.shape, r_head_k)

    w_head_q, w_head_k, w_head_v = torch.chunk(w_heads, 3, dim=-1)

klen = w_head_k.size(0)

w_head_q = w_head_q.view(qlen, bsz, self.n_head, self.d_head)           # qlen x bsz x n_head x d_head
w_head_k = w_head_k.view(klen, bsz, self.n_head, self.d_head)           # qlen x bsz x n_head x d_head
w_head_v = w_head_v.view(klen, bsz, self.n_head, self.d_head)           # qlen x bsz x n_head x d_head

r_head_k = r_head_k.view(rlen, self.n_head, self.d_head)                # qlen x n_head x d_head

#### compute attention score
rw_head_q = w_head_q + r_w_bias 
# print('rw_head_q',rw_head_q)                                        # qlen x bsz x n_head x d_head
# print('w head k', w_head_k)
# # print('rw bias', r_w_bias)
AC = torch.einsum('ibnd,jbnd->ijbn', (rw_head_q, w_head_k))             # qlen x klen x bsz x n_head

rr_head_q = w_head_q + r_r_bias
BD = torch.einsum('ibnd,jnd->ijbn', (rr_head_q, r_head_k))              # qlen x klen x bsz x n_head
# print('rr_head_q', rr_head_q[:3])
print('r_head_k', r_head_k[:3])
# print('BD ', BD[:3])
# print(BD[0] - BD[1])
# print(BD[0][0] - BD[0][1])
BD = self._rel_shift(BD)
# print('BD ', BD)

# [qlen x klen x bsz x n_head]
attn_score = AC + BD
attn_score.mul_(self.scale)
print('attn score ', attn_score)

#### compute attention probability
if attn_mask is not None and attn_mask.any().item():
    if attn_mask.dim() == 2:
        attn_score = attn_score.float().masked_fill(
            attn_mask[None,:,:,None], -float('inf')).type_as(attn_score)
    elif attn_mask.dim() == 3:
        attn_score = attn_score.float().masked_fill(
            attn_mask[:,:,:,None], -float('inf')).type_as(attn_score)

# [qlen x klen x bsz x n_head]
attn_prob = F.softmax(attn_score, dim=1)
attn_prob = self.dropatt(attn_prob)

#### compute attention vector
attn_vec = torch.einsum('ijbn,jbnd->ibnd', (attn_prob, w_head_v))

# [qlen x bsz x n_head x d_head]
attn_vec = attn_vec.contiguous().view(
    attn_vec.size(0), attn_vec.size(1), self.n_head * self.d_head)

##### linear projection
attn_out = self.o_net(attn_vec)
attn_out = self.drop(attn_out)

if self.pre_lnorm:
    ##### residual connection
    output = w + attn_out
else:
    ##### residual connection + layer normalization
    output = self.layer_norm(w + attn_out)

AttributeError: 'MemTransformerLM' object has no attribute 'pre_lnorm'

In [33]:
AC

NameError: name 'AC' is not defined

In [8]:
out = model(src[:, :2], tgt[:24, :2].contiguous())

mem_tokens, enc, mems = out[0], out[1], out[2:]

r_head_k tensor([[[ 0.2262, -0.5403,  0.0405,  ...,  0.0798,  0.0777,  1.1830],
         [ 0.5002, -0.2644,  0.0691,  ..., -0.0247, -0.0708,  0.0897],
         [-0.3503,  0.0289, -0.5878,  ..., -0.2117, -0.1172, -0.1238],
         [ 0.5241, -0.5015, -0.0266,  ...,  0.2789,  0.2498, -0.1229]],

        [[ 0.2521, -0.5876, -0.0805,  ...,  0.0289,  0.2788,  1.2227],
         [ 0.6034, -0.1618,  0.0834,  ..., -0.1337, -0.1154,  0.1049],
         [-0.2763, -0.2468, -0.5324,  ..., -0.1908, -0.2773, -0.0356],
         [ 0.5368, -0.3575, -0.0864,  ...,  0.3076,  0.2850, -0.1621]],

        [[ 0.3549, -0.5847, -0.0754,  ..., -0.1469,  0.3942,  1.1786],
         [ 0.6535,  0.0195,  0.0049,  ..., -0.2342, -0.1317,  0.0346],
         [-0.1697, -0.4524, -0.5574,  ..., -0.1915, -0.4983,  0.0562],
         [ 0.5492, -0.2864, -0.1481,  ...,  0.2096,  0.4107, -0.1843]]],
       grad_fn=<SliceBackward>)
attn score  tensor([[[[ 4.5926e-01, -7.4895e-01,  4.0674e-01,  1.8538e-01],
          [ 4.5926e-01, -

/home/ayd98/Desktop/MIPT/TXL/pytorch/mem_transformer.py:279: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1104.)
  attn_score = attn_score.float().masked_fill(


In [9]:
1/0

ZeroDivisionError: division by zero

In [ ]:
model.mem_tokens

In [ ]:
def _forward(self, dec_inp, mems=None, mem_tokens=None):

        word_emb = self.word_emb(dec_inp)

        mlen = mems[0].size(0) if mems is not None else 0
        
        # Concat with mem_tokens
        if mem_tokens is not None:
            word_emb = torch.cat((mem_tokens.detach(), word_emb), dim=0)
        elif self.num_mem_tokens not in (0, None):
            mem_tokens = self.mem_tokens.reshape(self.num_mem_tokens, 1, -1).repeat(1, dec_inp.shape[1], 1)
            word_emb = torch.cat((mem_tokens.detach(), word_emb), dim=0)

        # qlen, bsz = dec_inp.size()
        qlen = word_emb.shape[0]
        klen = mlen + qlen
        if self.same_length:
            all_ones = word_emb.new_ones(qlen, klen)
            mask_len = klen - self.mem_len
            if mask_len > 0:
                mask_shift_len = qlen - mask_len
            else:
                mask_shift_len = qlen
            dec_attn_mask = (torch.triu(all_ones, 1+mlen)
                    + torch.tril(all_ones, -mask_shift_len)).byte()[:, :, None] # -1
        else:
            dec_attn_mask = torch.triu(
                word_emb.new_ones(qlen, klen), diagonal=1+mlen).byte()[:,:,None]
        
        hids = []
        if self.attn_type == 0: # default
            pos_seq = torch.arange(klen-1, -1, -1.0, device=word_emb.device, 
                                   dtype=word_emb.dtype)
            if self.clamp_len > 0:
                pos_seq.clamp_(max=self.clamp_len)
            pos_emb = self.pos_emb(pos_seq)

            core_out = self.drop(word_emb)
            pos_emb = self.drop(pos_emb)
            print(len(core_out), [c.shape for c in core_out])

            hids.append(core_out)
            for i, layer in enumerate(self.layers):
                mems_i = None if mems is None else mems[i]
                core_out = layer(core_out, pos_emb, self.r_w_bias,
                        self.r_r_bias, dec_attn_mask=dec_attn_mask, mems=mems_i)
                hids.append(core_out)
        elif self.attn_type == 1: # learnable
            core_out = self.drop(word_emb)
            hids.append(core_out)
            for i, layer in enumerate(self.layers):
                if self.clamp_len > 0:
                    r_emb = self.r_emb[i][-self.clamp_len :]
                    r_bias = self.r_bias[i][-self.clamp_len :]
                else:
                    r_emb, r_bias = self.r_emb[i], self.r_bias[i]

                mems_i = None if mems is None else mems[i]
                core_out = layer(core_out, r_emb, self.r_w_bias[i],
                        r_bias, dec_attn_mask=dec_attn_mask, mems=mems_i)
                hids.append(core_out)
        elif self.attn_type == 2: # absolute
            pos_seq = torch.arange(klen - 1, -1, -1.0, device=word_emb.device,
                                   dtype=word_emb.dtype)
            if self.clamp_len > 0:
                pos_seq.clamp_(max=self.clamp_len)
            pos_emb = self.pos_emb(pos_seq)

            core_out = self.drop(word_emb + pos_emb[-qlen:])

            hids.append(core_out)
            for i, layer in enumerate(self.layers):
                mems_i = None if mems is None else mems[i]
                if mems_i is not None and i == 0:
                    mems_i += pos_emb[:mlen]
                core_out = layer(core_out, dec_attn_mask=dec_attn_mask,
                                 mems=mems_i)
                hids.append(core_out)
        elif self.attn_type == 3:
            core_out = self.drop(word_emb)

            hids.append(core_out)
            for i, layer in enumerate(self.layers):
                mems_i = None if mems is None else mems[i]
                if mems_i is not None and mlen > 0:
                    cur_emb = self.r_emb[i][:-qlen]
                    cur_size = cur_emb.size(0)
                    if cur_size < mlen:
                        cur_emb_pad = cur_emb[0:1].expand(mlen-cur_size, -1, -1)
                        cur_emb = torch.cat([cur_emb_pad, cur_emb], 0)
                    else:
                        cur_emb = cur_emb[-mlen:]
                    mems_i += cur_emb.view(mlen, 1, -1)
                core_out += self.r_emb[i][-qlen:].view(qlen, 1, -1)

                core_out = layer(core_out, dec_attn_mask=dec_attn_mask,
                                 mems=mems_i)
                hids.append(core_out)

        core_out = self.drop(core_out)

        new_mems = self._update_mems(hids, mems, mlen, qlen)

        return core_out, new_mems

In [ ]:
 _forward(model, src[:, :1])

In [ ]:
out, mems = _forward(model, src[:, :1])
out.shape

In [ ]:
out[:9]

In [ ]:
mem_tokens[0][0]

In [ ]:
mem_tokens[0][0] - mem_tokens[0][1]

In [ ]:
mem.shape

In [ ]:
mem.shape, pred_hid.shape, tgt.contiguous().shape

In [ ]:
loss = model.crit(pred_hid.view(-1, pred_hid.size(-1)), tgt.contiguous().view(-1))
loss = loss.view(tgt_len, -1)

In [ ]:
src.shape, out.shape

In [ ]:
class TransformerXL(nn.Module):
    def __init__(self, enc_kwargs, dec_kwargs):
        super().__init__()
        
        self.Encoder = MemTransformerLM(**enc_kwargs)
        self.Decoder = MemTransformerLM(**dec_kwargs)

        self.enc_kwargs = enc_kwargs
        self.dec_kwargs = dec_kwargs

    def forward(self, src, tgt, mems=None):
        enc, mems = self.Encoder._forward(src, mems=mems)

        num_mem = self.enc_kwargs['num_mem_tokens']
        if num_mem not in (None, 0):
            mem, enc = enc[:num_mem], enc[num_mem:]

        if self.dec_kwargs['mem_len'] == 0:
            mems = None


        start_tokens = tgt[:1, :]
        out, new_mems = self.Decoder._forward(start_tokens)
        
        # loss = self.crit(pred_hid.view(-1, pred_hid.size(-1)), target.view(-1))




In [ ]:
model.layers

In [ ]:
out.shape

In [ ]:
tgt.contiguous().view(-1)

In [ ]:
# rep = src[0].repeat(16, 1)
# hidden, new_mems = model._forward(rep)

mems = None
# dec_inp = src[0].repeat(21, 1).T
dec_inp = src[:21].T
self = model


In [ ]:

word_emb = self.word_emb(dec_inp)

mlen = mems[0].size(0) if mems is not None else 0

# Concat with mem_tokens
if self.num_mem_tokens not in (0, None):
    # memory = self.mem_tokens.repeat(1, dec_inp.shape[0], 1).clone()
    memory = self.mem_tokens.reshape(self.num_mem_tokens, 1, -1).repeat(1, dec_inp.shape[1], 1)
    word_emb = torch.cat((memory, word_emb), dim=0)


In [ ]:

# qlen, bsz = dec_inp.size()
qlen = word_emb.shape[0]
klen = mlen + qlen
if self.same_length:
    all_ones = word_emb.new_ones(qlen, klen)
    mask_len = klen - self.mem_len
    if mask_len > 0:
        mask_shift_len = qlen - mask_len
    else:
        mask_shift_len = qlen
    dec_attn_mask = (torch.triu(all_ones, 1+mlen)
            + torch.tril(all_ones, -mask_shift_len)).byte()[:, :, None] # -1
else:
    dec_attn_mask = torch.triu(
        word_emb.new_ones(qlen, klen), diagonal=1+mlen).byte()[:,:,None]

hids = []
if self.attn_type == 0: # default
    pos_seq = torch.arange(klen-1, -1, -1.0, device=word_emb.device, 
                            dtype=word_emb.dtype)
    if self.clamp_len > 0:
        pos_seq.clamp_(max=self.clamp_len)
    pos_emb = self.pos_emb(pos_seq)

    core_out = self.drop(word_emb)
    pos_emb = self.drop(pos_emb)

    hids.append(core_out)
    for i, layer in enumerate(self.layers):
        mems_i = None if mems is None else mems[i]
        core_out = layer(core_out, pos_emb, self.r_w_bias,
                self.r_r_bias, dec_attn_mask=dec_attn_mask, mems=mems_i)
        hids.append(core_out)
elif self.attn_type == 1: # learnable
    core_out = self.drop(word_emb)
    hids.append(core_out)
    for i, layer in enumerate(self.layers):
        if self.clamp_len > 0:
            r_emb = self.r_emb[i][-self.clamp_len :]
            r_bias = self.r_bias[i][-self.clamp_len :]
        else:
            r_emb, r_bias = self.r_emb[i], self.r_bias[i]

        mems_i = None if mems is None else mems[i]
        core_out = layer(core_out, r_emb, self.r_w_bias[i],
                r_bias, dec_attn_mask=dec_attn_mask, mems=mems_i)
        hids.append(core_out)
elif self.attn_type == 2: # absolute
    pos_seq = torch.arange(klen - 1, -1, -1.0, device=word_emb.device,
                            dtype=word_emb.dtype)
    if self.clamp_len > 0:
        pos_seq.clamp_(max=self.clamp_len)
    pos_emb = self.pos_emb(pos_seq)

    core_out = self.drop(word_emb + pos_emb[-qlen:])

    hids.append(core_out)
    for i, layer in enumerate(self.layers):
        mems_i = None if mems is None else mems[i]
        if mems_i is not None and i == 0:
            mems_i += pos_emb[:mlen]
        core_out = layer(core_out, dec_attn_mask=dec_attn_mask,
                            mems=mems_i)
        hids.append(core_out)
elif self.attn_type == 3:
    core_out = self.drop(word_emb)

    hids.append(core_out)
    for i, layer in enumerate(self.layers):
        mems_i = None if mems is None else mems[i]
        if mems_i is not None and mlen > 0:
            cur_emb = self.r_emb[i][:-qlen]
            cur_size = cur_emb.size(0)
            if cur_size < mlen:
                cur_emb_pad = cur_emb[0:1].expand(mlen-cur_size, -1, -1)
                cur_emb = torch.cat([cur_emb_pad, cur_emb], 0)
            else:
                cur_emb = cur_emb[-mlen:]
            mems_i += cur_emb.view(mlen, 1, -1)
        core_out += self.r_emb[i][-qlen:].view(qlen, 1, -1)

        core_out = layer(core_out, dec_attn_mask=dec_attn_mask,
                            mems=mems_i)
        hids.append(core_out)

core_out = self.drop(core_out)

new_mems = self._update_mems(hids, mems, mlen, qlen)

In [ ]:
hidden.shape, pred_hid.shape, mem_tokens.shape

In [ ]:
target.view(-1)

In [ ]:
target.shape

In [ ]:
pred_hid.view(-1, pred_hid.size(-1))

In [ ]:
src, tgt

In [ ]:
tgt_len = 49
target = tgt.contiguous()

hidden, new_mems = self._forward(src, mems=mems)

pred_hid = hidden[-(tgt_len+self.num_mem_tokens):]

if self.num_mem_tokens not in (0, None):
    mem_tokens, pred_hid = pred_hid[:self.num_mem_tokens].clone(), pred_hid[self.num_mem_tokens:].clone()
loss = self.crit(pred_hid.view(-1, pred_hid.size(-1)), target.view(-1))
loss = loss.view(tgt_len, -1)

In [ ]:
pred_hid.shape, target.shape


In [ ]:
pred_hid.view(-1, pred_hid.size(-1)).shape, target.view(-1).shape

In [ ]:
pos_seq.shape

In [ ]:
dec_inp.shape

In [ ]:
pos_emb.shape

In [ ]:
word_emb.shape

In [ ]:
core_out.shape, core_out.isnan().sum()

In [ ]:
# model.layers[0](hids[0],pos_emb,  self.r_w_bias,
#                 self.r_r_bias, dec_attn_mask=dec_attn_mask, mems=mems_i)

In [ ]:
new_mems

In [ ]:
mlen, qlen, klen

In [ ]:
pos_emb.shape, pos_seq.shape

In [ ]:
core_out.shape, core_out

In [ ]:
core_out[0][0] - core_out[1][0]

In [ ]:
pos_emb

In [ ]:
core_out[:, 0] - core_out[:, 1]

In [ ]:
hidden[0][0] - hidden[0][8]

In [ ]:
hidden[0]

In [ ]:
# class TransformerXL(nn.Module):
#     def __init__(self, enc_kwargs, dec_kwargs):
#         super().__init__()
        
#         self.Encoder = MemTransformerLM(**enc_kwargs)
#         self.Decoder = MemTransformerLM(**dec_kwargs)

#         self.enc_kwargs = enc_kwargs
#         self.dec_kwargs = dec_kwargs

#     def forward(self, src, tgt, mems=None):
#         hidden, mems = self.Encoder(src, )


In [ ]:
src.device

In [ ]:
model.mem_tokens.shape, model.mem_tokens.repeat(2, 1, 1).shape

In [ ]:
src.shape, tgt.shape

In [ ]:
model.layers

In [ ]:
model(src, tgt, )

In [ ]:

src.shape, src.repeat((1, 1, 2)).shape

In [ ]:
next(model.parameters()).device

In [ ]:
LEARNING_RATE = 0.0007

model_parameters = ParameterGrid({'dim': [128],
    'tie_token_embeds': [True],
    'return_tgt_loss': [True],
    'enc_num_tokens': [NUM_TOKENS],
    'depth,heads': [(2, 4)],
    'enc_max_seq_len': [24],
    'dec_num_tokens': [NUM_TOKENS],
    'dec_max_seq_len': [DEC_SEQ_LEN],
    'enc_num_memory_tokens': [2, 8, 0]})

print('Total runs: ', NUM_INITS * len(model_parameters))

In [ ]:
gen_train = data_loader(path=f'data{INPUT_LEN}', task_name=f'{TASK_NAME}_train', batch_size=BATCH_SIZE)
gen_val = data_loader(path=f'data{INPUT_LEN}', task_name=f'{TASK_NAME}_val', batch_size=VAL_SIZE)
gen_test = data_loader(path=f'data{INPUT_LEN}', task_name=f'{TASK_NAME}_test', batch_size=TEST_SIZE)


print_file = f'logs/{TASK_NAME}_{TAG}_memory_logs.txt'
t = time.time()
with torch.cuda.device(0):
    for init_num in range(NUM_INITS):
        with open(print_file, 'a') as f:
            f.write('\n\nInit number ' + str(init_num)+'\n')
        for i, param in enumerate(list(model_parameters)):
            with open(print_file, 'a') as f:
                f.write('\n\n' + str(param)+'\n')
            param['enc_depth'], param['enc_heads'] = param['depth,heads']
            param['dec_depth'], param['dec_heads'] = param['depth,heads']
            param.pop('depth,heads')

            with open(print_file, 'a') as f:
                f.write(f'{i / len(model_parameters) * 100}%')
            model = TransformerXL
            model = XTransformer(**param).cuda()

            model_name = f"{TASK_NAME}{INPUT_LEN}_dim{param['dim']}d{param['enc_depth']}h{param['enc_heads']}M{param['enc_num_memory_tokens']}l{param['enc_max_seq_len']}_{TAG}_v{init_num}"

            optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
            train_validate_model(model, 
                            train_generator=gen_train, 
                            val_generator=gen_val, 
                            optim=optim, 
                            model_name=model_name, 
                            config=param,
                            num_batches=NUM_BATCHES,
                            generate_every=GENERATE_EVERY,
                            print_file=print_file,
                            tag=TAG,
                            overfit_stop=False)
            test_model(model, gen_test, model_name, param, TASK_NAME, tag=TAG)
            with open(print_file, 'a') as f:
                f.write(f'\nTotal time: {time.time() - t}\n')
            t = time.time()